# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,59.11,41,0,12.66,CL,1741797952
1,1,adamstown,-25.0660,-130.1015,76.35,61,85,12.10,PN,1741798114
2,2,flying fish cove,-10.4217,105.6791,78.48,94,100,5.75,CX,1741798115
3,3,grytviken,-54.2811,-36.5092,33.84,76,65,15.08,GS,1741798116
4,4,nampa,43.5407,-116.5635,48.79,53,100,5.75,US,1741797863


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
# %%capture --no-display

# Configure the map plot
map_plot =city_data_df.hvplot.scatter(
    x="Lng",
    y="Lat",
    size="Humidity",
    color="Humidity",
    colormap="viridis",
    colorbar=True,
    title="City Humidity",
    frame_height=500,
    frame_width=700,
    hover_cols=["City", "Country"]
)

# Display the map
map_plot  



:Scatter   [Lng]   (Lat,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 80)
    & (city_data_df["Max Temp"] > 70)
    & (city_data_df["Wind Speed"] < 10)
    & (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,deoria,26.5047,83.7872,77.00,21,0,7.29,IN,1741798251
159,159,cabo san lucas,22.8909,-109.9124,76.75,33,0,1.99,MX,1741797856
246,246,teknaf,20.8624,92.3058,74.19,83,0,7.07,BD,1741798473
277,277,mazatlan,23.2167,-106.4167,74.91,39,0,4.00,MX,1741798508
475,475,fort bragg,35.1390,-79.0060,74.93,37,0,5.75,US,1741798721


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
49,deoria,IN,26.5047,83.7872,
159,cabo san lucas,MX,22.8909,-109.9124,
246,teknaf,BD,20.8624,92.3058,
277,mazatlan,MX,23.2167,-106.4167,
475,fort bragg,US,35.1390,-79.0060,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:

#Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "type": "hotel",
    "key": geoapify_key     
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}"
    params["bias"] = "proximity"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?categories=accommodation"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address_json = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
deoria - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
teknaf - nearest hotel: No hotel found
mazatlan - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
colgong - nearest hotel: No hotel found


,City,Country,Lat,Lng,Hotel Name
49,deoria,IN,26.5047,83.7872,No hotel found
159,cabo san lucas,MX,22.8909,-109.9124,No hotel found
246,teknaf,BD,20.8624,92.3058,No hotel found
277,mazatlan,MX,23.2167,-106.4167,No hotel found
475,fort bragg,US,35.1390,-79.0060,No hotel found
581,colgong,IN,25.2667,87.2167,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.scatter(
    x="Lng",
    y="Lat",
    size=200,
    color="Hotel Name",
    hover_cols=["Hotel Name"],
    title="Nearest Hotel",
    frame_height=500,
    frame_width=700
)

# Display the map
hotel_map_plot  

:Scatter   [Lng]   (Lat,Hotel Name)